import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.preprocess import clean_fraud_data, merge_geolocation, ip_to_int

# Load data
df = pd.read_csv('../data/raw/Fraud_Data.csv')
ip_df = pd.read_csv('../data/raw/IpAddress_to_Country.csv')  # Assume loaded

print(df.shape)
print(df.head())

In [1]:
# Clean the data using the preprocess module
df_clean = clean_fraud_data(df.copy())

# Quick checks post-cleaning
print(f"After cleaning: {df_clean.shape}")
print("\nData types:")
print(df_clean.dtypes)
print("\nClass distribution (fraud rate):")
print(df_clean['class'].value_counts(normalize=True))
print("\nMissing values summary:")
print(df_clean.isnull().sum().sum())  

NameError: name 'clean_fraud_data' is not defined

In [ ]:
df_clean = clean_fraud_data(df.copy())
print(f"After cleaning: {df_clean.shape}")
print(df_clean.info())
print(df_clean['class'].value_counts(normalize=True))  # Class imbalance

# Distributions
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
sns.histplot(df_clean['purchase_value'], kde=True, ax=axes[0,0])
axes[0,0].set_title('Purchase Value Distribution')

sns.histplot(df_clean['age'], kde=True, ax=axes[0,1])
axes[0,1].set_title('Age Distribution')

df_clean['source'].value_counts().plot(kind='bar', ax=axes[1,0])
axes[1,0].set_title('Source')

df_clean['browser'].value_counts().plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('Browser')
plt.tight_layout()
plt.show()

In [ ]:
# Fraud by source
sns.countplot(data=df_clean, x='source', hue='class')
plt.title('Fraud by Source')
plt.show()

# Age vs Class
sns.boxplot(data=df_clean, x='class', y='age')
plt.title('Age by Class')
plt.show()

# Correlation
corr = df_clean[['purchase_value', 'age', 'class']].corr()
sns.heatmap(corr, annot=True)
plt.title('Correlation Heatmap')
plt.show()

# Imbalance viz
plt.figure(figsize=(6,4))
df_clean['class'].value_counts().plot(kind='bar')
plt.title('Class Distribution (Imbalance: 5.4% Fraud)')
plt.ylabel('Count')
plt.show()

In [ ]:
df_geo = merge_geolocation(df_clean, ip_df)
print(f"Merged shape: {df_geo.shape}")
print(df_geo['country_code'].value_counts().head())

df_geo.to_parquet('../data/processed/fraud_geo.parquet')

# Class imbalance plot
plt.figure(figsize=(8, 5))
class_counts = df_clean['class'].value_counts()
colors = ['lightblue', 'salmon']  # Non-fraud blue, fraud red
bars = plt.bar(['Non-Fraud (0)', 'Fraud (1)'], class_counts.values, color=colors)

# Add counts on bars
for bar, count in zip(bars, class_counts.values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 100, 
             f'{count:,}', ha='center', va='bottom', fontweight='bold')

plt.title('Transaction Class Distribution\n(Overall Fraud Rate: 5.4%)', fontsize=14, fontweight='bold')
plt.ylabel('Number of Transactions')
plt.xlabel('Class')
plt.ylim(0, max(class_counts.values) * 1.05)
plt.grid(axis='y', alpha=0.3)

# Annotate fraud rate
fraud_rate = df_clean['class'].mean() * 100
plt.text(0.5, max(class_counts.values) * 0.8, f'Fraud Rate: {fraud_rate:.1f}%', 
         ha='center', va='center', transform=plt.gca().transAxes, 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

plt.tight_layout()
plt.show()

print(f"Imbalance Ratio: {class_counts[0]:,} legit vs. {class_counts[1]:,} fraud (1:{class_counts[0]//class_counts[1]:.0f})")

In [4]:
# Quick clean for ip_df (if not done)
ip_df_clean = ip_df.drop_duplicates(subset=['lower_bound_ip_address', 'upper_bound_ip_address'])
print(f"IP ranges after dedup: {ip_df_clean.shape}")

# Merge geolocation
df_geo = merge_geolocation(df_clean, ip_df_clean)
print(f"Merged dataset shape: {df_geo.shape}")
print(f"Successful mappings: {(df_geo['country_code'] != 'Unknown').mean():.1%}")
print("\nTop countries:")
print(df_geo['country_code'].value_counts().head(10))

# Save intermediate merged data
df_geo.to_parquet('../data/processed/fraud_geo_merged.parquet')
print("\nSaved: fraud_geo_merged.parquet")

NameError: name 'ip_df' is not defined

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('default')
sns.set_palette("husl")

# Load data
df_cc = pd.read_csv('../data/raw/creditcard.csv')
print(f"Dataset shape: {df_cc.shape}")
print(df_cc.head())
print("\nTarget distribution:")
print(df_cc['Class'].value_counts(normalize=True))

ModuleNotFoundError: No module named 'matplotlib'

# Fraud rate by country
country_fraud = df_geo.groupby('country_code')['class'].agg(['mean', 'count'])
country_fraud['mean'] *= 100  # Percentage
top_fraud_countries = country_fraud.sort_values('mean', ascending=False).head(10)

sns.barplot(data=top_fraud_countries.reset_index(), x='country_code', y='mean')
plt.title('Top 10 Fraud Rates by Country (%)')
plt.xticks(rotation=45)
plt.show()

print("High-risk countries:", top_fraud_countries.index.tolist())